In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import os
import sys
sys.path.append('../src')
from torch import nn
from omegaconf import OmegaConf
from torch.utils.data import DataLoader
import torch
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
import numpy as np
from tqdm import tqdm
pd.options.display.max_columns=100

In [25]:
from datalib import VentilatorData,VentilatorData2,VentilatorDataClassification,VentilatorDataMultiLabel
import modellib
from utils import fc
import datalib
from train_classification import get_group_dict

In [15]:
DATA_DIR = '/mnt/disks/extra_data/kaggle/ventilator_prediction/'
R_MAP = {5: 0, 50: 1, 20: 2}
C_MAP = {20: 0, 50: 1, 10: 2}
RC_MAP = {'2050': 0,
 '2020': 1,
 '5020': 2,
 '5050': 3,
 '550': 4,
 '520': 5,
 '5010': 6,
 '2010': 7,
 '510': 8}

In [4]:
train = pd.read_csv(os.path.join(DATA_DIR,'train.csv'))
test = pd.read_csv(os.path.join(DATA_DIR,'test.csv'))
train.breath_id = train.breath_id.map(dict(zip(train.breath_id.unique().tolist(),range(train.breath_id.nunique()))))
test.breath_id = train.breath_id.map(dict(zip(train.breath_id.unique().tolist(),range(train.breath_id.nunique()))))

In [5]:
train.head()

,id,breath_id,R,C,time_step,u_in,u_out,pressure
0,1,0,20,50,0.000000,0.083334,0,5.837492
1,2,0,20,50,0.033652,18.383041,0,5.907794
2,3,0,20,50,0.067514,22.509278,0,7.876254
3,4,0,20,50,0.101542,22.808822,0,11.742872
4,5,0,20,50,0.135756,25.355850,0,12.234987


In [6]:
train['RC'] = train['R'].astype('str')+train['C'].astype('str')

In [7]:
target1 = train[['breath_id','RC']].drop_duplicates()

In [16]:
target1.RC = target1.RC.map(RC_MAP)

In [17]:
target1.iloc[0]

breath_id    0
RC           0
Name: 0, dtype: int64

In [23]:
%%time
train_df = fc(train[['id','breath_id','time_step','u_out','u_in','pressure']],include_R_C=False)

../src/utils.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["mean_{}_last_{}".format("u_in", i)] = (
../src/utils.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["min_{}_last_{}".format("u_in", i)] = (


5
10
5
10
CPU times: user 1min 18s, sys: 8.1 s, total: 1min 26s
Wall time: 1min 26s


In [24]:
train_df.head()

,id,breath_id,time_step,u_out,u_in,pressure,mean_u_in_last_5,min_u_in_last_5,max_u_in_last_5,std_u_in_last_5,mean_u_in_last_10,min_u_in_last_10,max_u_in_last_10,std_u_in_last_10,mean_u_in_next_5,min_u_in_next_5,max_u_in_next_5,std_u_in_next_5,mean_u_in_next_10,min_u_in_next_10,max_u_in_next_10,std_u_in_next_10,u_in_cumsum,u_in_cummean,u_in_cummax,lag_u_in_1,lead_u_in_1,lag_u_in_2,lead_u_in_2,lag_u_in_3,lead_u_in_3,lag_u_in_4,lead_u_in_4,auc,lag_auc_1,lead_auc_1,lag_auc_2,lead_auc_2,per_change_u_in_lag_u_in_1,per_change_u_in_lead_u_in_1,per_change_u_in_lag_u_in_2,per_change_u_in_lead_u_in_2,per_change_u_in_lag_u_in_3,per_change_u_in_lead_u_in_3,per_change_u_in_lag_u_in_4,per_change_u_in_lead_u_in_4,per_change_auc_lag_auc_1,per_change_auc_lead_auc_1,per_change_auc_lag_auc_2,per_change_auc_lead_auc_2
79,1,0,0.000000,0,0.083334,5.837492,0.083334,0.083334,0.083334,0.000000,0.083334,0.083334,0.083334,0.000000,17.828065,0.083334,25.355850,10.229525,22.651316,0.083334,28.313036,8.515846,0.083334,0.083334,0.083334,0.000000,18.383041,0.000000,22.509278,0.000000,22.808822,0.000000,25.355850,0.000000,0.000000,0.310718,0.000000,0.692350,0.999988,-219.592083,0.999988,-269.105925,0.999988,-272.700387,0.999988,-303.264110,0.000000,-310718.055487,0.000000,-692350.186987
78,2,0,0.033652,0,18.383041,5.907794,9.233188,0.083334,18.383041,12.939847,9.233188,0.083334,18.383041,12.939847,23.263372,18.383041,27.259866,3.351989,25.329658,18.383041,28.313036,3.151678,18.466375,9.233188,18.383041,0.083334,22.509278,0.000000,22.808822,0.000000,25.355850,0.000000,27.259866,0.310718,0.000000,0.692350,0.000000,0.771038,0.995467,-0.224459,1.000000,-0.240753,1.000000,-0.379307,1.000000,-0.482881,0.999997,-1.228222,0.999997,-1.481467
77,3,0,0.067514,0,22.509278,7.876254,13.658551,0.083334,22.509278,11.936136,13.658551,0.083334,22.509278,11.936136,25.012260,22.509278,27.259866,2.278378,26.167635,22.509278,28.313036,2.004828,40.975653,13.658551,22.509278,18.383041,22.808822,0.083334,25.355850,0.000000,27.259866,0.000000,27.127486,0.692350,0.310718,0.771038,0.000000,0.823936,0.183313,-0.013308,0.996298,-0.126462,1.000000,-0.211050,1.000000,-0.205169,0.551212,-0.113653,0.999999,-0.190057
76,4,0,0.101542,0,22.808822,11.742872,15.946119,0.083334,22.808822,10.766279,15.946119,0.083334,22.808822,10.766279,25.871951,22.808822,27.259866,1.872664,26.716034,22.808822,28.313036,1.602636,63.784476,15.946119,22.808822,22.509278,25.355850,18.383041,27.259866,0.083334,27.127486,0.000000,26.807732,0.771038,0.692350,0.823936,0.310718,0.892947,0.013133,-0.111669,0.194038,-0.195146,0.996346,-0.189342,1.000000,-0.175323,0.102054,-0.068607,0.597013,-0.158110
75,5,0,0.135756,0,25.355850,12.234987,17.828065,0.083334,25.355850,10.229525,17.828065,0.083334,25.355850,10.229525,26.883130,25.355850,27.864715,0.935871,27.114142,25.355850,28.313036,0.834682,89.140326,17.828065,25.355850,22.808822,27.259866,22.509278,27.127486,18.383041,26.807732,0.083334,27.864715,0.823936,0.771038,0.892947,0.692350,0.924862,0.100451,-0.075092,0.112265,-0.069871,0.274998,-0.057260,0.996713,-0.098946,0.064202,-0.083757,0.159704,-0.122491


In [101]:
train.shape

(6036000, 9)

In [26]:
%%time
train_grp_data = get_group_dict(train_df)

100%|██████████| 75450/75450 [07:20<00:00, 171.24it/s]

CPU times: user 7min 15s, sys: 9.61 s, total: 7min 25s
Wall time: 7min 20s


In [102]:
data = VentilatorDataMultiLabel(
        group_dict = train_grp_data,
        label_df = target1.iloc[:1000],
        categorical_columns = ['u_out'],
        numerical_columns = ['time_step', 'u_out', 'u_in', 'pressure',
       'mean_u_in_last_5', 'min_u_in_last_5', 'max_u_in_last_5',
       'std_u_in_last_5', 'mean_u_in_last_10', 'min_u_in_last_10',
       'max_u_in_last_10', 'std_u_in_last_10', 'mean_u_in_next_5',
       'min_u_in_next_5', 'max_u_in_next_5', 'std_u_in_next_5',
       'mean_u_in_next_10', 'min_u_in_next_10', 'max_u_in_next_10',
       'std_u_in_next_10', 'u_in_cumsum', 'u_in_cummean', 'u_in_cummax',
       'lag_u_in_1', 'lead_u_in_1', 'lag_u_in_2', 'lead_u_in_2', 'lag_u_in_3',
       'lead_u_in_3', 'lag_u_in_4', 'lead_u_in_4', 'auc', 'lag_auc_1',
       'lead_auc_1', 'lag_auc_2', 'lead_auc_2', 'per_change_u_in_lag_u_in_1',
       'per_change_u_in_lead_u_in_1', 'per_change_u_in_lag_u_in_2',
       'per_change_u_in_lead_u_in_2', 'per_change_u_in_lag_u_in_3',
       'per_change_u_in_lead_u_in_3', 'per_change_u_in_lag_u_in_4',
       'per_change_u_in_lead_u_in_4', 'per_change_auc_lag_auc_1',
       'per_change_auc_lead_auc_1', 'per_change_auc_lag_auc_2',
       'per_change_auc_lead_auc_2'],
        target_column='pressure',

)

dl = DataLoader(
    dataset=data,
    shuffle=True,
    num_workers=1,
    batch_size=8,
    pin_memory=True,
)

In [107]:
from train_multilabel import train_model

In [117]:
%%time
a,b = train_model('../configs/lstm-multi-label-classification.yaml',fold_nums=[0])

5
10
5
10
['time_step', 'u_out', 'u_in', 'pressure', 'mean_u_in_last_5', 'min_u_in_last_5', 'max_u_in_last_5', 'std_u_in_last_5', 'mean_u_in_last_10', 'min_u_in_last_10', 'max_u_in_last_10', 'std_u_in_last_10', 'mean_u_in_next_5', 'min_u_in_next_5', 'max_u_in_next_5', 'std_u_in_next_5', 'mean_u_in_next_10', 'min_u_in_next_10', 'max_u_in_next_10', 'std_u_in_next_10', 'u_in_cumsum', 'u_in_cummean', 'u_in_cummax', 'lag_u_in_1', 'lead_u_in_1', 'lag_u_in_2', 'lead_u_in_2', 'lag_u_in_3', 'lead_u_in_3', 'lag_u_in_4', 'lead_u_in_4', 'auc', 'lag_auc_1', 'lead_auc_1', 'lag_auc_2', 'lead_auc_2', 'per_change_u_in_lag_u_in_1', 'per_change_u_in_lead_u_in_1', 'per_change_u_in_lag_u_in_2', 'per_change_u_in_lead_u_in_2', 'per_change_u_in_lag_u_in_3', 'per_change_u_in_lead_u_in_3', 'per_change_u_in_lag_u_in_4', 'per_change_u_in_lead_u_in_4', 'per_change_auc_lag_auc_1', 'per_change_auc_lead_auc_1', 'per_change_auc_lag_auc_2', 'per_change_auc_lead_auc_2']


100%|██████████| 75450/75450 [06:12<00:00, 202.44it/s] 


(2414400, 53) (603600, 53)
CPU times: user 7min 13s, sys: 18.5 s, total: 7min 31s
Wall time: 8min 4s


In [118]:
for x in a:
    pass

In [116]:
# train_grp_data[40231]

In [103]:
m = next(iter(dl))

In [106]:
for x in dl:
    pass

In [42]:
m['num'].shape,m['pressure'].shape,m['rc'].shape

(torch.Size([8, 80, 48]), torch.Size([8, 80]), torch.Size([8, 1]))

In [48]:
target1['RC'].nunique()

9

In [84]:
config = OmegaConf.load('../configs/lstm-multi-label-classification.yaml')

In [85]:
config.model.kwargs['output_dim'] = 100

In [89]:
mod = getattr(modellib,config.model['class'])(config.model['kwargs'])

{'init_type': 'yakama'}
tensor(189.0285)
tensor(-66.8056)


In [91]:
# mod

In [92]:
preds = mod(m)

In [97]:
preds['pred_pressure'].shape,preds['pred_pressure'].shape

torch.Size([8, 80, 100])

In [75]:
m['cat'].shape

torch.Size([8, 80, 1])

In [60]:
rnn = nn.LSTM(input_size = 48,hidden_size=32,bidirectional=True,batch_first=True)

In [63]:
a,_ = rnn(m['num'])

In [66]:
nn.AdaptiveAvgPool1d(output_size=1)(a.permute(0,2,1)).shape

torch.Size([8, 64, 1])

In [59]:
m['num'].shape

torch.Size([8, 80, 48])

In [100]:
preds['pred_RC'].shape

torch.Size([8, 9])